In [ ]:
#pip install git+https://github.com/oneconvergence/dkube.git@2.2

In [ ]:
import os, time
from dkube.sdk import *

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

Creating Repos

In [ ]:
code_name = generate("dkube-examples")
code = DkubeCode(username, name=code_name)
code.update_git_details("https://github.com/oneconvergence/dkube-examples.git")
api.create_code(code)

In [ ]:
dataset_name = generate("mnist")
dataset = DkubeDataset(username, name=dataset_name)
dataset.update_git_details("https://dkube.s3.amazonaws.com/datasets/mnist.pkl.gz")
dataset.update_dataset_source(source='pub_url')
#dataset.update_awss3_details(bucket="mybucket",prefix="datasets/mnist.pkl.gz",key="access_key",secret="access_secret")
api.create_dataset(dataset)

In [ ]:
model_name = generate("mnist")
model = DkubeModel(username, name=model_name)
model.update_model_source(source='dvs')
api.create_model(model)

Train

In [ ]:
training_name = generate("mnist-sdk-run")
training = DkubeTraining(username, name=training_name)
training.update_container(framework="tensorflow_2.0.0", image_url="docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3")
training.add_code(code_name)
training.update_startupscript("python mnist/train.py")
training.add_input_dataset(dataset_name, mountpath="/mnist")
training.add_output_model(model_name, mountpath="/model")
current_version = api.get_model_latest_version (username, model_name)['uuid']
api.create_training_run(training)
#wait while model version is updated
while current_version == api.get_model_latest_version (username, model_name)['uuid']:
    time.sleep(5)

Serve

In [ ]:
deployment_name = generate("mnist-serv")
serving = DkubeServing(username, name=deployment_name)
serving.set_transformer(transformer=True, script="mnist/transformer.py")
serving.update_transformer_code(code=code_name)
serving.update_transformer_image(image_url="docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3")
serving.update_serving_model(model_name)
serving.update_serving_image(image_url="ocdr/tensorflowserver:2.0.0")
#serving.set_production_deploy()
serving.update_autoscaling_config(min_replicas=1, max_concurrent_requests=1)
api.create_test_inference(serving)

Deployment URL

In [ ]:
deployments = api.list_model_deployments(username)
for deployment in deployments:
    if deployment_name == deployment["name"]:
        params = deployment["parameters"]
        if params['generated']['status']['state'] == 'RUNNING':
            print("Serving URL:",params['generated']['details']['serving']['servingurl'])